In [1]:
from __future__ import annotations

import json
import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RefMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
async def dummy_loader(name: str) -> dict:
    await sleep(0.2)
    return {
        'name': name,
        'age': 20
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RefMap(
            ref=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RefMap(
                    ref=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RefMap(
                            ref=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RefMap(
                    ref=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    rel=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [7]:
with cProfile.Profile() as pr:
  db = DataBase()
  rec = await data_source.load([data], db=db)

  stats = pstats.Stats(pr)

In [8]:
print(db[Search].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_Search".result_count AS result_count
FROM "test_db_schema_Search"


In [9]:
print(db[Search.term].select_str)

SELECT "test_db_schema_Search".term AS term
FROM "test_db_schema_Search"


In [10]:
print(db[Search.result_count].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_Search".result_count AS result_count
FROM "test_db_schema_Search"


In [11]:
print(db[Search.results].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_Project".number AS number,
       "test_db_schema_Project".status AS status,
       "test_db_schema_Project"."end" AS "end",
       "test_db_schema_Project".start AS
START, "test_db_schema_Project".name AS name
FROM "test_db_schema_Search"
JOIN "test_db_schema_SearchResult" ON "test_db_schema_SearchResult"."test_db_schema_Search_term" = "test_db_schema_Search".term
JOIN "test_db_schema_Project" ON "test_db_schema_SearchResult"."test_db_schema_Project_number" = "test_db_schema_Project".number


In [12]:
db[Search.term].values()

['test']

In [15]:
list(db[Search.result_count].items())

[('test', 3)]

In [16]:
db[Project].values()

[]

In [14]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         103563 function calls (97694 primitive calls) in 0.253 seconds

   Ordered by: cumulative time
   List reduced from 2069 to 120 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.046    0.023 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:4355(_update_dict)
        1    0.000    0.000    0.034    0.034 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3523(engine)
        2    0.000    0.000    0.024    0.012 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:830(to_df)
    40/25    0.000    0.000    0.020    0.001 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1694(__get__)
        3    0.000    0.000    0.011    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:688(_load_records)
       27    0.000    0.000    0.009    0.000 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2337(_has_t